# Explore, annotate, and analyze multi-dimensional images in Python with napari

## 1.1 – a *fast* 2D viewer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import napari

In [2]:
import tifffile

image = tifffile.imread(
    '/Users/jni/projects/demos/spatialdata-sandbox/'
    'visium_io/data/Visium_Mouse_Olfactory_Bulb_image.tif'
)

In [3]:
image.shape

(10000, 10000, 4)

In [4]:
%matplotlib qt

In [5]:
plt.imshow(image)

In [6]:
viewer, layer = napari.imshow(image)

# 2024-11-06 14:58:05,130 INFO OpenGL.acceleratesupport acceleratesupport.py:17 -- No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'



## 1.2 – a *multidimensional* viewer

### 3D multichannel cells

In [9]:
from skimage import data

cells = data.cells3d()

cells.shape

(60, 2, 256, 256)

In [10]:
data.cells3d?

Signature: data.cells3d()
Docstring:
3D fluorescence microscopy image of cells.

The returned data is a 3D multichannel array with dimensions provided in
``(z, c, y, x)`` order. Each voxel has a size of ``(0.29 0.26 0.26)``
micrometer. Channel 0 contains cell membranes, channel 1 contains nuclei.

Returns
-------
cells3d: (60, 2, 256, 256) uint16 ndarray
    The volumetric images of cells taken with an optical microscope.

Notes
-----
The data for this was provided by the Allen Institute for Cell Science.

It has been downsampled by a factor of 4 in the row and column dimensions
to reduce computational time.

The microscope reports the following voxel spacing in microns:

    * Original voxel size is ``(0.290, 0.065, 0.065)``.
    * Scaling factor is ``(1, 4, 4)`` in each dimension.
    * After rescaling the voxel size is ``(0.29 0.26 0.26)``.
File:      ~/micromamba/envs/amms/lib/python3.12/site-packages/skimage/data/_fetchers.py
Type:      function

In [11]:
viewer, layer = napari.imshow(
        cells,
        channel_axis=1,
        scale=[0.29, 0.26, 0.26],
        )

### CryoET Dynamo PCA analysis

Credit: Alister Burt (currently at Genentech)

In [12]:
from pathlib import Path
from functools import cached_property

import pandas as pd
import numpy as np
import mrcfile


class DPCAA:
    def __init__(self, eigenvolumes_dir, eigentable_file):
        self.eigenvolumes_dir = Path(eigenvolumes_dir)
        self.eigentable_file = eigentable_file
    
    @cached_property
    def eigenvolumes(self):
        volume_files = list(self.eigenvolumes_dir.glob('*.mrc'))
        df = pd.DataFrame({'path' : volume_files}).sort_values(by='path')
        df['eigenvolume'] = df['path'].apply(lambda x: mrcfile.open(x).data)
        eigenvolumes = np.stack(df['eigenvolume'])
        return eigenvolumes

    @cached_property
    def eigentable(self):
        return np.loadtxt(self.eigentable_file, delimiter=',')

    def spectral_average_from_coefficients(self, coefficients, normalise=True):
        coefficients = coefficients.squeeze()[..., np.newaxis, np.newaxis, np.newaxis]
        spectral_average = (coefficients * self.eigenvolumes).sum(axis=-4)

        if normalise:
            spectral_average = self._normalise_volume(spectral_average)

        return spectral_average

    def spectral_average_from_idx(self, idx):
        """generate spectral average from particles at idx
        """
        coefficients = self._coefficients_from_idx(idx)
        return self.spectral_average_from_coefficients(coefficients)

    def _coefficients_from_idx(self, idx):
        """generate coefficients from a set of particles at idx
        """
        return self.eigentable[idx, :].sum(axis=0)

    def _generate_volume_series(self, eig, n_bins=10, qcut=True):
        eig_coefficients = self.eigentable[:, eig]

        if qcut:
            cut = pd.qcut(eig_coefficients, n_bins)
        else:
            cut = pd.cut(eig_coefficients, n_bins)

        volumes = [self.spectral_average_from_idx(cut == subset) for subset in cut.categories]
        return np.stack(volumes)

    def _generate_volume_series_vectorised(self, eig, n_bins=10, qcut=True):
        eig_coefficients = self.eigentable[:, eig]

        if qcut:
            cut = pd.qcut(eig_coefficients, n_bins)
        else:
            cut = pd.cut(eig_coefficients, n_bins)

        coefficients = np.stack(
            [self._coefficients_from_idx(cut == subset) for subset in cut.categories]
        )
        volumes = self.spectral_average_from_coefficients(coefficients)
        return volumes

    def _normalise_volume(self, volume):
        """independently normalise a stack of volumes to mean 0 standard deviation 1
        """
        volume_axes = (-1, -2, -3)
        volume_mean = np.expand_dims(volume.mean(axis=volume_axes), axis=volume_axes)
        volume_std = np.expand_dims(volume.std(axis=volume_axes), axis=volume_axes)
        return (volume - volume_mean) / volume_std


In [13]:
from pathlib import Path

folder = Path('WM4196')
eigenvolumes = folder / 'eigenvolumes'
eigentable = folder / 'eigentable.csv'

pca = DPCAA(eigenvolumes_dir=eigenvolumes, eigentable_file=eigentable)

viewer = napari.Viewer()

n_bins = 10

volumes = np.stack([
        pca._generate_volume_series(comp, n_bins=n_bins, qcut=True)
        for comp in range(50)
        ])

viewer.add_image(volumes)

<Image layer 'volumes' at 0x37d57e300>

## 2 – a *layered* viewer

### overlay images, segmentations, point detections, and more

In [14]:
from skimage import data

coins = data.coins()[50:-50, 50:-50]

viewer, im_layer = napari.imshow(coins)

In [15]:
from skimage import filters, measure, morphology, segmentation

thresholded = filters.threshold_otsu(coins) < coins
closed = morphology.closing(thresholded, morphology.square(4))
no_border = segmentation.clear_border(closed)
cleaned = morphology.remove_small_objects(no_border, 20)

segmented = measure.label(cleaned).astype(np.uint8)

label_layer = viewer.add_labels(segmented)

In [16]:
centroids = np.array([p.centroid for p in measure.regionprops(segmented)])
pts_layer = viewer.add_points(centroids, size=5)

(TODO: We need a cool shapes layer demo)

### cryoET particle picking refinement

Credit: Alister Burt (currently at Genentech)

Code: https://github.com/alisterburt/napari-cryo-et-demo  
Data: https://www.ebi.ac.uk/empiar/EMPIAR-10164/

In [19]:
import mrcfile

# files containing data
tomogram_file = '/Users/jni/data/napari-cryo-et-demo/hiv/01_10.00Apx.mrc'
particles_file = '/Users/jni/data/napari-cryo-et-demo/hiv/01_10.00Apx_particles.star'

# loading data into memory
# tomogram is a numpy array containing image array data
with mrcfile.open(tomogram_file) as mrc:
    tomogram = mrc.data.copy()

viewer, tomo_layer = napari.imshow(
        tomogram,
        blending='translucent_no_depth',
        colormap='gray_r',
        )

![sphere picking](static/sphere-annotator.gif)

![sphere fitting](https://teamtomo.org/_images/hiv-oversampling1.png)

from https://teamtomo.org/walkthroughs/EMPIAR-10164/geometrical-picking.html

In [20]:
import starfile
from scipy.spatial.transform import Rotation as R

# df is a pandas DataFrame containing table of info from STAR file
# including positions and orientations
df = starfile.read(particles_file)

# get particle positions as (n, 3) numpy array from DataFrame
zyx = df[
        ['rlnCoordinateZ', 'rlnCoordinateY', 'rlnCoordinateX']
        ].to_numpy()

pts_layer = viewer.add_points(
        zyx,
        face_color='cornflowerblue',
        size=10,
        )

In [21]:
# get particle orientations as Euler angles from DataFrame
euler_angles = df[
        ['rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi']
        ].to_numpy()

# turn Euler angles into a scipy 'Rotation' object, rotate Z vectors to see
# where they point for the aligned particle
rotations = R.from_euler(
        seq='ZYZ', angles=euler_angles, degrees=True
        ).inv()
direction_xyz = rotations.apply([0, 0, 1])
direction_zyx = direction_xyz[:, ::-1]

# set up napari vectors layer data
# (n, 2, 3) array
# dim 0: batch dimension
# dim 1: first row is start point of vector,
#        second is direction vector
# dim 2: components of direction vector e.g. (z, y, x)

vectors = np.stack((zyx, direction_zyx), axis=1)

vec_layer = viewer.add_vectors(
        vectors, length=10, edge_color='orange'
        )

![reconstruction](https://teamtomo.org/_images/result2.png)

from https://teamtomo.org/walkthroughs/EMPIAR-10164/m.html

In [22]:
pts_data = pts_layer.data

In [23]:
type(pts_data)

numpy.ndarray

## 3 – an *annotation* and *proofreading* tool

### interactive segmentation of 3D cells

Semi-automated methods in Python.

In [24]:
viewer, (membrane_layer, nuclei_layer) = napari.imshow(
        cells,
        channel_axis=1,
        name=['membrane', 'nuclei'],
        )

In [25]:
# grab individual channels and convert to float in [0, 1]

membranes = cells[:, 0, :, :] / np.max(cells)
nuclei = cells[:, 1, :, :] / np.max(cells)

In [26]:
from skimage import filters


edges = filters.farid(nuclei)

edges_layer = viewer.add_image(
        edges,
        blending='additive',
        colormap='yellow',
        )

In [27]:
from scipy import ndimage as ndi

denoised = ndi.median_filter(nuclei, size=3)

In [28]:
li_thresholded = denoised > filters.threshold_li(denoised)

threshold_layer = viewer.add_image(
        li_thresholded,
        opacity=0.3,
        )

In [29]:
from skimage import morphology

width = 20

holes_removed = morphology.remove_small_holes(
        li_thresholded, width ** 3
        )

speckle_removed = morphology.remove_small_objects(
        holes_removed, width ** 3
        )

viewer.layers[-1].visible = False

viewer.add_image(
        speckle_removed,
        name='cleaned',
        opacity=0.3,
        );

In [30]:
from skimage import measure

labels = measure.label(speckle_removed)

viewer.layers[-1].visible = False
viewer.add_labels(
        labels,
        opacity=0.5,
        blending='translucent_no_depth'
        )


<Labels layer 'labels' at 0x393ce3590>

In [31]:
# Sean's solution
from scipy import ndimage as ndi
from skimage.feature import peak_local_max

spacing = [0.29, 0.26, 0.26]
distances = ndi.distance_transform_edt(
    speckle_removed, sampling=spacing
)
dt_smoothed = filters.gaussian(distances, sigma=5)
peaks = peak_local_max(dt_smoothed, min_distance=5)

pts_layer = viewer.add_points(
        peaks,
        name="sean's points",
        size=4,
        n_dimensional=True,  # points have 3D "extent"
        )

In [32]:
points_data = pts_layer.data
points_data

array([[ 35.        ,  98.        , 160.        ],
       [ 36.        ,  13.        , 152.        ],
       [ 35.        , 158.        , 110.        ],
       [ 35.        , 219.        ,  84.        ],
       [ 34.        , 184.        ,  49.        ],
       [ 33.        , 237.        , 129.        ],
       [ 34.        ,  71.        , 111.        ],
       [ 35.        , 146.        , 243.        ],
       [ 36.        ,  47.        , 182.        ],
       [ 36.        ,  34.        ,  76.        ],
       [ 35.        ,  11.        ,  21.        ],
       [ 38.        , 140.        ,  38.        ],
       [ 35.        ,  51.        , 231.        ],
       [ 32.        , 202.        , 169.        ],
       [ 35.        , 143.        , 192.        ],
       [ 41.        ,  78.        ,  59.        ],
       [ 41.        ,  98.        ,  49.        ],
       [ 36.        , 103.92105263, 251.87246964],
       [ 36.        , 220.52024291, 249.02226721],
       [ 36.        , 249.54048

In [33]:
from skimage import segmentation, util

markers = util.label_points(points_data, nuclei.shape)
markers_big = morphology.dilation(markers, morphology.ball(5))

segmented = segmentation.watershed(
        edges, markers_big, mask=speckle_removed,
        )

seg_layer = viewer.add_labels(
        segmented, blending='translucent_no_depth',
        )

viewer.layers['labels'].visible = False

## 4.1 – a *lazy* viewer

Tribolium castaneum light sheet microscopy data from the [Cell tracking challenge](http://celltrackingchallenge.net/3d-datasets/) contributed by Akanksha Jain, MPI-CBG Dresden.

In [34]:
import zarr

image = zarr.open('/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr/0/')

print(f'{image.nbytes / 1e9:.0f}GB')

213GB


In [35]:
print(image.shape)

(60, 988, 1868, 964)


In [36]:
print(image.chunks)

(1, 64, 512, 512)


In [37]:
viewer, layer = napari.imshow(image)

## 4.2 – lazy annotation 🦥🎨, thank you zarr! 🧊❤️🙏

In [38]:
type(image), image.shape, image.nbytes / 1e9

(zarr.core.Array, (60, 988, 1868, 964), 213.49715712)

In [44]:
viewer = napari.Viewer()
layer_multi = viewer.add_image(
        image,
        rendering='attenuated_mip',
        name='tribolium',
        contrast_limits=(1000, 6000),
        )

labels = zarr.open(
        '/Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr',
        dtype=np.uint32,
        shape=image.shape,
        write_empty_chunks=False,
        chunks=image.chunks,
        )

In [40]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/

.                     01_GT                 random.mov
..                    02                    random.png
.DS_Store             02_GT                 random2.png
01                    anim-random.py        random3.png
01-labels.zarr        anim.py               random4.png
01.ome.zarr           backup-anim-random.py tribolium.mov


In [43]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

ls: /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr: No such file or directory


In [45]:
labels.shape

(60, 988, 1868, 964)

In [46]:
layer = viewer.add_labels(labels)

INFO: Calculating empty label on non-numpy array is not supported


In [47]:
!ls -a /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

.        ..       .zarray  29.7.2.0


In [48]:
!rm -rf /Users/jni/data/Fluo-N3DL-TRIF/01-labels.zarr

## 5 – plays well with others

napariboard

In [49]:
!python /Users/jni/projects/napariboard-proto/napariboard.py

## 6 – extensible with plugins

## napari-ome-zarr

In [1]:
viewer = napari.Viewer()

# 2024-07-18 11:48:18,930 INFO OpenGL.acceleratesupport acceleratesupport.py:17 -- No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'



In [2]:
viewer.open(
        '/Users/jni/data/Fluo-N3DL-TRIF/01.ome.zarr',
        plugin='napari-ome-zarr',
        )

# 2024-07-18 11:48:23,329 INFO ome_zarr.reader reader.py:175 -- root_attr: multiscales

# 2024-07-18 11:48:23,330 INFO ome_zarr.reader reader.py:175 -- root_attr: omero

# 2024-07-18 11:48:23,330 INFO ome_zarr.reader reader.py:298 -- datasets [{'coordinateTransformations': [{'scale': [90, 0.38, 0.38, 0.38], 'type': 'scale'}, {'translation': [0, 0, 0, 0], 'type': 'translation'}], 'path': '0'}, {'coordinateTransformations': [{'scale': [90, 0.74, 0.74, 0.74], 'type': 'scale'}, {'translation': [0, 0.19, 0.19, 0.19], 'type': 'translation'}], 'path': '1'}, {'coordinateTransformations': [{'scale': [90, 1.48, 1.48, 1.48], 'type': 'scale'}, {'translation': [0, 0.95, 0.95, 0.95], 'type': 'translation'}], 'path': '2'}]

# 2024-07-18 11:48:23,331 INFO ome_zarr.reader reader.py:306 -- resolution: 0

# 2024-07-18 11:48:23,331 INFO ome_zarr.reader reader.py:312 --  - shape ('t', 'z', 'y', 'x') = (60, 988, 1868, 964)

# 2024-07-18 11:48:23,331 INFO ome_zarr.reader reader.py:313 --  - chunks =  ['1', '

[<Image layer 'tribolium' at 0x323592900>]

napari-pdf-reader (I shit you not 😂)

In [50]:
viewer = napari.Viewer()

pdf_layer, = viewer.open('data/project_jupyter.pdf', plugin='napari-pdf-reader')

/Users/jni/micromamba/envs/amms/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:199: UserWarning: data shape (8, 3300, 2550, 3) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.
  warnings.warn(


In [51]:
from skimage import color

pdfbw = color.rgb2gray(pdf_layer.data)
pdf_layer.visible = False
pdfbw_layer = viewer.add_image(
        pdfbw[:, ::2, ::2],
        scale=(2, 2, 2),
        rendering='translucent',
        )
viewer.dims.ndisplay = 3

In [52]:
from magicgui import magicgui, widgets

@magicgui(
        shear={'widget_type': widgets.FloatSlider,
               'min': 0,
               'max': pdfbw.shape[1]},
        auto_call=True,
        )
def set_layer_xz_shear(shear: float):
    pdfbw_layer.affine = [
            [1    , 0, 0, 0],
            [0    , 1, 0, 0],
            [shear, 0, 1, 0],
            [0    , 0, 0, 1],
            ]

dw = viewer.window.add_dock_widget(set_layer_xz_shear);

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~/micromamba/envs/amms/lib/python3.12/site-packages/napari/_qt/threads/status_checker.py:100, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
     97     return
     99 try:
--> 100     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(3.5, 1649.5, 1274.5), zoom=0.11328186188931992, angles=(-2.2063077529990824, 72.66401632205257, 89.50976640648655), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(-1220.3332793384097, 706.1974823590747, 701.0588870354782), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=3, ndisplay=3, order=(0, 1, 2), axis_labels=('0', '1', '2'), rollable=(True, True, True), range=(RangeTuple(start=0.0, stop=14.0, step=1.0), RangeTuple(start=0.0, stop=3299.0, step=1.0), Ran

Traceback (most recent call last):
  File "/Users/jni/micromamba/envs/amms/lib/python3.12/site-packages/napari/_qt/threads/status_checker.py", line 86, in run
    self.calculate_status()
  File "/Users/jni/micromamba/envs/amms/lib/python3.12/site-packages/napari/_qt/threads/status_checker.py", line 110, in calculate_status
    self.status_and_tooltip_changed.emit(res)
                                         ^^^
UnboundLocalError: cannot access local variable 'res' where it is not associated with a value
